In [1]:
!pip install pyro-ppl

    100% |████████████████████████████████| 491kB 28.1MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 28.3MB/s ta 0:00:01
    79% |█████████████████████████▎      | 595.3MB 39.1MB/s eta 0:00:05  1% |▋                               | 13.4MB 38.0MB/s eta 0:00:20    2% |▊                               | 16.2MB 35.9MB/s eta 0:00:21    6% |██▏                             | 52.1MB 50.5MB/s eta 0:00:14    27% |█████████                       | 209.8MB 36.7MB/s eta 0:00:15    30% |█████████▊                      | 229.4MB 44.6MB/s eta 0:00:12    36% |███████████▋                    | 273.6MB 40.3MB/s eta 0:00:12    42% |█████████████▌                  | 317.3MB 35.5MB/s eta 0:00:13    55% |█████████████████▉              | 420.6MB 32.0MB/s eta 0:00:11    56% |██████████████████              | 424.6MB 47.0MB/s eta 0:00:07    56% |██████████████████▏             | 426.9MB 27.7MB/s eta 0:00:12    66% |█████████████████████▏          | 498.4MB 37.8MB/s eta 0:00:07

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    81% |██████████████████████████▏     | 616.6MB 42.3MB/s eta 0:00:04

    100% |████████████████████████████████| 753.4MB 16kB/s  eta 0:00:01    89% |████████████████████████████▊   | 674.9MB 34.9MB/s eta 0:00:03    89% |████████████████████████████▊   | 677.1MB 32.6MB/s eta 0:00:03    91% |█████████████████████████████▏  | 686.1MB 40.9MB/s eta 0:00:02
  Running setup.py bdist_wheel for opt-einsum ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
Successfully built opt-einsum
fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
%matplotlib inline
# python libraties
import os, itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image

# pytorch libraries
import torch
import torch.nn.functional as F
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# sklearn libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# data directory 
data_dir = 'data'   

In [3]:
def set_parameter_requires_grad(model, finetuning = False):
    '''
        This function freezes the parameters of the model.
        
        Parameters:
            model: model to be freezed before training
            finetuning: if finetuning is true then the model will not be freezed
        
        Returns:
            Nothing
    '''
    if not finetuning:
        print(finetuning)
        for param in model.parameters():
            param.requires_grad = False

In [7]:
class FCModel(nn.Module):
    def __init__(self, num_ftrs, hidden_ftrs1, hidden_ftrs2, output_ftrs):
        
        super(FCModel, self).__init__()
        self.fc1 = nn.Linear(num_ftrs, hidden_ftrs1)
        self.fc2 = nn.Linear(hidden_ftrs1, hidden_ftrs2)
        self.fc3 = nn.Linear(hidden_ftrs2, output_ftrs)
        self.drop1 = nn.Dropout(0.2)
        self.drop2 = nn.Dropout(0.2)
        
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = self.fc3(x)
        return self.softmax(x)
        

In [8]:
def initialize_models(model_name, num_classes, fine_tuning = False):
    '''
        This function initialize3s the pretrained model
    '''
    
    model_fe = None
    input_size = 0
    
    if(model_name == 'resnet'):
        model_fe = models.resnet50(pretrained = True)
        set_parameter_requires_grad(model_fe, fine_tuning)
        num_ftrs = model_fe.fc.in_features
        hidden_ftrs1 = model_fe.fc.out_features
        hidden_ftrs2 = int(hidden_ftrs1 / 2)
        model_fe.fc = FCModel(num_ftrs, hidden_ftrs1, hidden_ftrs2, num_classes)
        input_size = 224
    
    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_fe, input_size
        

In [9]:
model, input_size = initialize_models('resnet', num_classes = 7)
print(input_size)
model

False
224


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
device = torch.device('cuda:0')
# Put the model on the device:
model = model.to(device)

In [11]:
mean = np.load('means.npz')
stdevs = np.load('stdevs.npz')
norm_mean = mean['arr_0']
norm_std = stdevs['arr_0']
print(norm_mean)
print(norm_std)

[0.7630329  0.54564583 0.5700466 ]
[0.14092815 0.15261224 0.1699708 ]


In [12]:
# transfromations for train images
train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(norm_mean, norm_std)])
# define the transformation of the test images.
test_transform = transforms.Compose([transforms.Resize((input_size,input_size)), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

In [13]:
class HamDataset(Dataset):
    def __init__(self, csvpath, transform = None):
        self.df = pd.read_csv(csvpath)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))
        
        if self.transform:
            X = self.transform(X)
        
        return X, y

def get_data_loader(csvpath, transform = test_transform, batch_size = 32):
    if csvpath == 'train.csv':
        transform = train_transform
    csvpath = os.path.join(data_dir, csvpath)
    dataset = HamDataset(csvpath, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    
    return dataloader

In [67]:
# train_dl = get_data_loader('train.csv')

In [14]:
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss().to(device)

In [15]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [19]:
total_loss_train = []
total_acc_train = []
def train(dataloader, model, criterion, optimizer, epoch):
    model.train() # set on training mode
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(dataloader)
    for i, data in enumerate(dataloader):
        # 1 get the data and bring it to proper format
        images, labels = data
        N = images.size(0) # for total no of images in one iter
#         print(images.shape)
        
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)
        
        # 2 Zero the parameter gradient
        optimizer.zero_grad()
        
        # 3 forward 
        outputs = model(images)
        
        # 4 loass calculate
        loss = criterion(outputs, labels)
        
        # 5 loss backward
        loss.backward()
        optimizer.step()
        
        prediction = outputs.max(1, keepdim = True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    
    return train_loss.avg, train_acc.avg
        
        

In [20]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [21]:
epoch_num = 10
best_val_acc = 0
total_loss_val, total_acc_val = [],[]
train_loader = get_data_loader('train.csv')
val_loader = get_data_loader('validation.csv')
# print(model)
for epoch in range(1, epoch_num+1):
    loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch)
    loss_val, acc_val = validate(val_loader, model, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

C:\ProgramData\Anaconda3\envs\pythongpu\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[epoch 1], [iter 100 / 901], [train loss 1.79382], [train acc 0.36156]
[epoch 1], [iter 200 / 901], [train loss 1.77887], [train acc 0.37562]
[epoch 1], [iter 300 / 901], [train loss 1.76349], [train acc 0.38875]
[epoch 1], [iter 400 / 901], [train loss 1.75578], [train acc 0.39727]
[epoch 1], [iter 500 / 901], [train loss 1.74227], [train acc 0.41225]
[epoch 1], [iter 600 / 901], [train loss 1.73178], [train acc 0.42391]
[epoch 1], [iter 700 / 901], [train loss 1.72143], [train acc 0.43464]
[epoch 1], [iter 800 / 901], [train loss 1.71212], [train acc 0.44480]
[epoch 1], [iter 900 / 901], [train loss 1.70385], [train acc 0.45344]
------------------------------------------------------------
[epoch 1], [val loss 1.49440], [val acc 0.66118]
------------------------------------------------------------
*****************************************************
best record: [epoch 1], [val loss 1.49440], [val acc 0.66118]
*****************************************************
[epoch 2], [iter 100

[epoch 10], [iter 400 / 901], [train loss 1.58136], [train acc 0.58031]
[epoch 10], [iter 500 / 901], [train loss 1.59014], [train acc 0.57137]
[epoch 10], [iter 600 / 901], [train loss 1.58798], [train acc 0.57380]
[epoch 10], [iter 700 / 901], [train loss 1.59088], [train acc 0.57094]
[epoch 10], [iter 800 / 901], [train loss 1.59042], [train acc 0.57152]
[epoch 10], [iter 900 / 901], [train loss 1.59046], [train acc 0.57139]
------------------------------------------------------------
[epoch 10], [val loss 1.42545], [val acc 0.73983]
------------------------------------------------------------


In [ ]:
state = {
    'epochs' : 10,
    'model state dict' : model.state_dict(),
    'optimizer state dict' : optimizer.state_dict()
#     'val accuracy': best_val_acc
}

torch.save(state, 'resnet50.pth')